Documentation:
https://python.langchain.com/v0.1/docs/use_cases/sql/agents/
https://python.langchain.com/v0.2/docs/integrations/tools/sql_database/
https://python.langchain.com/v0.2/docs/tutorials/sql_qa/#agents


In [10]:
!pip install langchain
!pip install langchain-core
!pip install langchain-community
!pip install openai
!pip install tiktoken
!pip install chromadb
!pip install pypdf
!pip install python-dotenv
!pip install -U langchain-openai
!pip install --upgrade --quiet langchainhub langgraph

In [11]:
import os
import pandas as pd
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from IPython.display import Markdown, HTML, display
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
import langchain
from langchain.callbacks.base import BaseCallbackHandler

In [12]:
chat_model = ChatOpenAI(model="gpt-4o", openai_api_key='')
summary_model = ChatOpenAI(model="gpt-4o-mini", openai_api_key='')

In [13]:
#database set up
import sqlite3

import requests
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool
from sqlalchemy import inspect


def get_engine_for_chinook_db():
    """Pull sql file, populate in-memory database, and create engine."""
    url = "https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql"
    response = requests.get(url)
    sql_script = response.text

    connection = sqlite3.connect(":memory:", check_same_thread=False)
    connection.executescript(sql_script)
    return create_engine(
        "sqlite://",
        creator=lambda: connection,
        poolclass=StaticPool,
        connect_args={"check_same_thread": False},
    )


engine = get_engine_for_chinook_db()

db = SQLDatabase(engine)

def show_schema(engine):
    inspector = inspect(engine)
    tables = inspector.get_table_names()

    for table in tables:
        print(f"\nSchema for table '{table}':")
        columns = inspector.get_columns(table)
        for column in columns:
            print(f"{column['name']} ({column['type']})")

# Call the function to display the schema
show_schema(engine)


Schema for table 'Album':
AlbumId (INTEGER)
Title (NVARCHAR(160))
ArtistId (INTEGER)

Schema for table 'Artist':
ArtistId (INTEGER)
Name (NVARCHAR(120))

Schema for table 'Customer':
CustomerId (INTEGER)
FirstName (NVARCHAR(40))
LastName (NVARCHAR(20))
Company (NVARCHAR(80))
Address (NVARCHAR(70))
City (NVARCHAR(40))
State (NVARCHAR(40))
Country (NVARCHAR(40))
PostalCode (NVARCHAR(10))
Phone (NVARCHAR(24))
Fax (NVARCHAR(24))
Email (NVARCHAR(60))
SupportRepId (INTEGER)

Schema for table 'Employee':
EmployeeId (INTEGER)
LastName (NVARCHAR(20))
FirstName (NVARCHAR(20))
Title (NVARCHAR(30))
ReportsTo (INTEGER)
BirthDate (DATETIME)
HireDate (DATETIME)
Address (NVARCHAR(70))
City (NVARCHAR(40))
State (NVARCHAR(40))
Country (NVARCHAR(40))
PostalCode (NVARCHAR(10))
Phone (NVARCHAR(24))
Fax (NVARCHAR(24))
Email (NVARCHAR(60))

Schema for table 'Genre':
GenreId (INTEGER)
Name (NVARCHAR(120))

Schema for table 'Invoice':
InvoiceId (INTEGER)
CustomerId (INTEGER)
InvoiceDate (DATETIME)
BillingAddr

In [89]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, just "
    "reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

messages = [
    HumanMessage(content="What country's customers spent the most money?"),
    AIMessage(content="USA Customers spent the most Money"),
]


print(contextualize_q_prompt.invoke(
    {
        "chat_history": messages,
        "input": "What album did that countries customers most buy?",
    }

))

answer = chat_model.invoke(contextualize_q_prompt.invoke(
    {
        "chat_history": messages,
        "input": "What album did that countries customers most buy?",
    }

))



messages=[SystemMessage(content='Given a chat history and the latest user question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is.'), HumanMessage(content="What country's customers spent the most money?"), AIMessage(content='USA Customers spent the most Money'), HumanMessage(content='What album did that countries customers most buy?')]


In [93]:
agent_description = """
You are an agent designed to interact with a SQL database.
## Instructions:
- Given an input question, create a syntactically correct SQL query
to run, then look at the results of the query and return the answer.
- Some Questions make refer to the answers of previous questions.
in this situation you should use the necessary context from the entire conversation
- Unless the user specifies a specific number of examples they wish to
obtain, **ALWAYS** limit your query to at most 1 result.
- Never query for all the columns from a specific table, only ask for
the relevant columns given the question.
- You have access to tools for interacting with the database.
- You MUST double check your query before executing it.If you get an error
while executing a query,rewrite the query and try again.
- DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.)
to the database.
- DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE, ONLY USE THE RESULTS
OF THE CALCULATIONS YOU HAVE DONE.
- ALWAYS, as part of your final answer, explain how you got to the answer
on a section that starts with: "Explanation:". Include the SQL query as
part of the explanation section.
- Only use the below tools. Only use the information returned by the
below tools to construct your query and final answer.
- Do not make up table names, only use the tables returned by any of the
tools below.
- If the question does not seem related to the database, just answer the question
without any tools.

"""

agent_format_instructions = MSSQL_AGENT_FORMAT_INSTRUCTIONS = """

## Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do.
Action: the action to take, should be one of [{tool_names}].
Action Input: the input to the action.
Observation: the result of the action.
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer.
Final Answer: the final answer to the original input question.
```
===> End of Example

"""
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=chat_model)
sql_agent = create_sql_agent(
    prefix=agent_description,
    llm=chat_model,
    format_instructions=agent_format_instructions,
    toolkit=toolkit,
    agent_type = "tool-calling",
    handle_parsing_errors=True,
    top_k=5,
    verbose=True,
)

In [94]:
def recontextQ(input_text, messages):
  contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, just "
    "reformulate it if needed and otherwise return it as is."
 )
  contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
  )

  new_question = summary_model.invoke(contextualize_q_prompt.invoke(
    {
        "chat_history": messages,
        "input": input_text,
    }
  ))

  return new_question.content



chat_messages = []
summary_messages = []

def run_agent_with_memory(input_text, chat_messages, summary_messages):

    new_question = recontextQ(input_text, summary_messages)
    response = sql_agent.run(new_question)
    messages.append(HumanMessage(content=input_text))
    messages.append(AIMessage(content=response))
    chat_messages.append(HumanMessage(content=new_question))
    chat_messages.append(AIMessage(content=response))
    return response

while True:
    query = input("You: ")
    if query.lower() == "exit":
        break
    response = run_agent_with_memory(query, chat_messages, summary_messages )
    print(f"Agent: {response}")


You: What country's customers spent the most money?


> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Customer, Invoice'}`



CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	LastName	Company	Address	City	State	Country	PostalCode	Phone	Fax	Email	SupportRepId
1	Luís	Gonçalves	Embraer - Empresa Brasileira de Aeron

KeyboardInterrupt: Interrupted by user